<a href="https://colab.research.google.com/github/Swayam0407/CourseCompass_Web/blob/main/Ranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries
!pip install pandas scikit-learn tabulate

# Import libraries
import pandas as pd
import numpy as np
from google.colab import files
from sklearn.preprocessing import MinMaxScaler
from tabulate import tabulate

# Prompt to upload the dataset
print("Please upload your dataset.")
uploaded = files.upload()

# Load the dataset
filename = next(iter(uploaded.keys()))
courses_df = pd.read_csv(filename)

# Ensure column names are correctly identified
print("Column names in the dataset:", courses_df.columns)



Please upload your dataset.


Saving sample_courses_dataset.csv to sample_courses_dataset (3).csv
Column names in the dataset: Index(['Course Title', 'Platform', 'Instructor Name', 'Category/Topic',
       'Rating', 'Number of Reviews', 'Enrollment Count', 'Completion Rate',
       'Instructor Reputation', 'Time Spent on Course Materials (hours)',
       'Update Frequency (months)', 'Certification (Yes/No)', 'Price ($)'],
      dtype='object')


In [ ]:
# Clean 'Certification (Yes/No)' column
if 'Certification (Yes/No)' in courses_df.columns:
    courses_df['Certification (Yes/No)'] = courses_df['Certification (Yes/No)'].astype(str).str.strip().str.upper()
    certification_map = {'YES': 1, 'NO': 0}
    courses_df['Certification (Yes/No)'] = courses_df['Certification (Yes/No)'].replace(certification_map).fillna(0)
else:
    print("Column 'Certification (Yes/No)' not found in the dataset.")

# Weights for the criteria
weights = {
    "Rating": 0.25,
    "Number of Reviews": 0.15,
    "Enrollment Count": 0.15,
    "Completion Rate": 0.10,
    "Instructor Reputation": 0.15,
    "Time Spent on Course Materials (hours)": 0.05,
    "Update Frequency (months)": 0.05,
    "Certification (Yes/No)": 0.05,
    "Price ($)": 0.05
}

# Normalize the columns
scaler = MinMaxScaler()
normalized_columns = ['Rating', 'Number of Reviews', 'Enrollment Count', 'Completion Rate',
                      'Instructor Reputation', 'Time Spent on Course Materials (hours)',
                      'Update Frequency (months)', 'Price ($)']

for column in normalized_columns:
    if column in courses_df.columns:
        courses_df[column] = scaler.fit_transform(courses_df[[column]])
    else:
        print(f"Column '{column}' not found in the dataset.")

# Adjust 'Update Frequency (months)' and 'Price ($)' as they should be inversely proportional
if 'Update Frequency (months)' in courses_df.columns:
    courses_df['Update Frequency (months)'] = 1 - courses_df['Update Frequency (months)']
if 'Price ($)' in courses_df.columns:
    courses_df['Price ($)'] = 1 - courses_df['Price ($)']

# Calculate composite score
courses_df['Composite Score'] = 0
for criterion, weight in weights.items():
    if criterion in courses_df.columns:
        courses_df['Composite Score'] += courses_df[criterion] * weight

# Rank the courses within each topic
courses_df['Rank'] = courses_df.groupby('Category/Topic')['Composite Score'].rank(ascending=False)

# Sort courses by topic and rank
courses_df = courses_df.sort_values(by=['Category/Topic', 'Rank'])


In [ ]:
# Function to get courses by topic
def get_courses_by_topic(topic):
    required_df = courses_df[courses_df['Category/Topic'].str.contains(topic, case=False, na=False)]
    if required_df.empty:
        print(f"No courses found for the topic: {topic}")
    else:
        print(f"Courses related to the topic: {topic}")
        print(tabulate(required_df[['Course Title', 'Category/Topic', 'Composite Score', 'Rank']], headers='keys', tablefmt='pretty'))

# User input
user_topic = input("Enter the topic to search for related courses: ").strip()
get_courses_by_topic(user_topic)


Enter the topic to search for related courses: finance
Courses related to the topic: finance
+----+------------------+----------------+---------------------+------+
|    |   Course Title   | Category/Topic |   Composite Score   | Rank |
+----+------------------+----------------+---------------------+------+
| 13 | Finance Course 2 |    Finance     | 0.5013174982911826  | 1.0  |
| 14 | Finance Course 3 |    Finance     | 0.49010423786739565 | 2.0  |
| 12 | Finance Course 1 |    Finance     | 0.3934461152882206  | 3.0  |
| 15 | Finance Course 4 |    Finance     | 0.37468386876281645 | 4.0  |
+----+------------------+----------------+---------------------+------+
